## SVM regression with PCA

In this tutorial, we are going to understand whta is SVM and PCA. These concepts are both inportand and have different uses seperately. However people are often confused by the two, today we use a simple example to clarify the differene between the two concepts.

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
data = pd.read_csv("./input/diabetes.csv")
print(data.shape)
data.head(5)

(768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
features=data.iloc[:,:-1]
labels=data.iloc[:,-1]
print(features.head(2))
print('--------')
print(labels.head(2))

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   

   DiabetesPedigreeFunction  Age  
0                     0.627   50  
1                     0.351   31  
--------
0    1
1    0
Name: Outcome, dtype: int64


## Priciple component analysis (PCA) 
PCA is a way to reduce the dimentionality of the system while capturing the features that represents the original system.

source: 
https://www.youtube.com/watch?v=MrtPbruYbWY&fbclid=IwAR3Swm01tc6MPM95ftGjY8fgU9r5CIy82bQwvoIbdjQj4onHNTbUJWRsn6w

https://www.kaggle.com/uciml/pima-indians-diabetes-database/downloads/diabetes.csv/notebook

In [9]:
#Test train split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size=0.2)

#Standard scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#Applying PCA here
from sklearn.decomposition import PCA
pca = PCA(n_components= None) #We will set it none so that we can see the variance explained and then choose no of comp.
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

explained_variance = pca.explained_variance_ratio_
explained_variance

array([0.26608124, 0.2125024 , 0.1292791 , 0.10800322, 0.09565838,
       0.08606059, 0.05414537, 0.0482697 ])

Now we have maped the original feature data to a new feature set with a linear transform. the number of features reamins unchanged. From the scores, listed in the array, we could choose the first n values so that our feature numbers are reduced to n features. Note that the reduced features are no longer the original features, because they have gone through a linear transformation.

Let's see compare the performance of a logistic regression model using the original features and the reduced features.

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')
# Without PCA

X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size=0.2)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

classifier = LogisticRegression(random_state=0)
classifier.fit(X_train,y_train)

y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test,y_pred)#Compute confusion matrix
print(cm)

# PCA

X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size=0.2)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

pca = PCA(n_components= 2)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)


classifier = LogisticRegression(random_state=0)# Create the classifier and train using training data
classifier.fit(X_train_reduced,y_train)

y_pred = classifier.predict(X_test_reduced)#Predict the test set values

cm_r = confusion_matrix(y_test,y_pred)#Compute confusion matrix
print(cm_r)

[[92 11]
 [19 32]]
[[86 17]
 [29 22]]


It is seen that there is no significant changes between the two feature sets(original and the linearly transformed and reduced version), which means that we have successfully reduced the dimentionality of the feature set.

## Support Vector Machine
SVM is a technique to find the best fit for the data points. Yet, this is not the only features of SVM. SVM also includes a a kernel which performs nonlinear tranform to the data points. On can say that with these kernels, the features are mapped to a higher dimention. To summerize, svm does not do any dimentional reduction, it further maps the original features to a higher dimention.

Source:
https://www.youtube.com/watch?v=Y6RRHw9uN9o&fbclid=IwAR2Z7_pkuJhxmSXcBvl9wwXrOai3bLkyg5xtY0GzMeQDH4lvrFgWnTZLiHI

## Support Vector Regression
Instead of trying to fit the largest possible gab between the two classes while limiting marginal violations, Support Vector Regression tries to fit as many istances as possible between the gap.

Source:
https://www.youtube.com/watch?v=IcCZaA7zITc

In [19]:
X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size=0.2)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#Create classifier object
from sklearn.svm import SVC
classifier_svm_kernel = SVC(C=5.0,kernel='rbf', gamma=0.12,tol=0.00001)
classifier_svm_kernel.fit(X_train,y_train)

#Predict the result for test values
y_pred = classifier_svm_kernel.predict(X_test)

#Compute confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

array([[77, 18],
       [26, 33]], dtype=int64)

#### Applying grid search for optimal parameters and model after k-fold validation

In [20]:
from sklearn.model_selection import GridSearchCV

parameters = [{'C':[0.01,0.1,1,10,50,100,500,1000], \
               'kernel':['rbf'], \
               'gamma': [0.1,0.125,0.15,0.17,0.2]}]
grid_search = GridSearchCV(estimator=classifier_svm_kernel, \
                           param_grid=parameters, \
                           scoring ='accuracy',\
                           cv=10,n_jobs=-1)
grid_search = grid_search.fit(X_train,y_train)

best_accuracy = grid_search.best_score_
opt_param = grid_search.best_params_

print(best_accuracy)
print(opt_param)

0.7817589576547231
{'C': 10, 'gamma': 0.125, 'kernel': 'rbf'}


#### Use the optimised hyperparameters to retrain the model with original features

In [21]:
X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size=0.2)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#Create classifier object
from sklearn.svm import SVC
classifier_svm_kernel = SVC(C=10.0,kernel='rbf', gamma=0.125,tol=0.00001)
classifier_svm_kernel.fit(X_train,y_train)

#Predict the result for test values
y_pred = classifier_svm_kernel.predict(X_test)

#Compute confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

array([[91, 13],
       [22, 28]], dtype=int64)

Now we do have a better performance

In [23]:
X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size=0.2)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

pca = PCA(n_components= 2)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)


#Create classifier object
from sklearn.svm import SVC
classifier_svm_kernel = SVC(C=10.0,kernel='rbf', gamma=0.125,tol=0.00001)
classifier_svm_kernel.fit(X_train,y_train)

#Predict the result for test values
y_pred = classifier_svm_kernel.predict(X_test)

#Compute confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

array([[99, 13],
       [22, 20]], dtype=int64)

Wow, isnt it amazing, using only two components, we achieved a really similar performance. By only using two features, we are able to save a lot of time on computations.